In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

from tqdm import tqdm
tqdm.pandas(desc="Applying function")

from shapely.geometry import Point

In [2]:
gdf_buildings = gpd.read_file('../../data/processed/building/building_manhattan_commercial_street_traffic_density.geojson')

In [3]:
gdf_buildings.head()

,height,mpluto_bbl,cnstrct_yr,globalid,bin,office_area,retail_area,residential_area,OBJECTID,street_width_min,...,idw_aadt_2019,idw_aadt_2020,idw_aadt_2021,idw_atvc_2018,idw_atvc_2019,idw_atvc_2020,office_within_450ft,retail_within_450ft,residential_within_450ft,geometry
0,59.722628,1021210037,1910,{A0E56BCC-A86B-4CEF-9A42-9B4ECD61743F},1062896,0,2250,10344,100007,60.0,...,13651.722938,15362.929888,17083.134233,110.061311,140.829357,332.410589,5550,59662,898416,"MULTIPOLYGON (((1001211.761 244524.798, 100123..."
1,155.49,1008870001,1935,{0B06F7A8-1F83-44F3-97DD-AC524A6374CA},1018457,50018,4000,0,101045,32.0,...,10065.445841,10282.943569,11495.144080,129.010552,178.320105,272.446791,2550406,351887,1164766,"MULTIPOLYGON (((989135.970 210787.603, 989181...."
2,167.21446831,1013490035,1931,{A091951D-C73E-4B77-9A61-716C666C5446},1039988,2890,0,70110,99804,60.0,...,14437.629737,14445.844421,16299.638335,151.952815,174.349496,240.654167,170032,135993,3123941,"MULTIPOLYGON (((994153.480 215659.003, 994165...."
3,28.57,1010610006,1910,{2BE9F674-67E7-4A49-8535-F636A16D8DA5},1026714,0,0,0,103068,32.0,...,10455.417903,10087.378012,11238.595524,163.402973,182.997568,237.267362,228739,75352,1677852,"MULTIPOLYGON (((986858.328 218070.430, 986839...."
4,73.13,1003060013,1900,{2403088E-77A6-4071-8B5E-D54AFBC720D9},1003956,0,3800,0,92425,34.0,...,7632.299988,7433.877764,8351.143163,105.344303,137.101029,350.242494,99564,185023,676931,"MULTIPOLYGON (((986180.396 200723.234, 986187...."


In [5]:
gdf_buildings.crs

<Projected CRS: EPSG:2263>
Name: NAD83 / New York Long Island (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: United States (USA) - New York - counties of Bronx; Kings; Nassau; New York; Queens; Richmond; Suffolk.
- bounds: (-74.26, 40.47, -71.8, 41.3)
Coordinate Operation:
- name: SPCS83 New York Long Island zone (US survey foot)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

## Distance from Park

In [6]:
gdf_parks = gpd.read_file('../../data/raw/Parks Properties_20240405.geojson').to_crs(gdf_buildings.crs)

In [7]:
gdf_buildings = gdf_buildings.sjoin_nearest(gdf_parks.loc[:,['geometry']],distance_col='distance_to_park').drop(columns='index_right')

## Distance from School

In [8]:
df_school = pd.read_csv('../../data/raw/2019_-_2020_School_Locations_20240405.csv')

In [9]:
points = [Point(x,y) for x,y in zip(df_school.loc[:, 'LONGITUDE'], df_school.loc[:, 'LATITUDE'])]
gdf_school = gpd.GeoDataFrame(df_school, geometry=points, crs=4326)
gdf_school = gdf_school.to_crs(gdf_buildings.crs)

In [10]:
gdf_buildings = gdf_buildings.sjoin_nearest(gdf_school.loc[:,['geometry']],distance_col='distance_to_school').drop(columns='index_right')

In [12]:
gdf_buildings = gdf_buildings.drop_duplicates(subset=['bin'])

In [13]:
gdf_buildings.to_file('../../data/processed/building/building_manhattan_commercial_street_traffic_density_other.geojson', driver='GeoJSON')

In [16]:
gdf_buildings.drop(columns=['geometry']).to_csv('../../data/processed/building/building_240405.csv', index=False)